In [1]:
import sys
sys.path.append(r'../')

In [2]:
from dolfin import *
import numpy as np

import cell_computation as com
import cell_geom as ce
import cell_material as ma
from copy import deepcopy

In [3]:
import logging
logging.getLogger('FFC').setLevel(logging.WARNING)

## Setting

In [4]:
mesh = Mesh(r"../m.xml")
# mesh = Mesh(r"m_fine.xml")

cell = ce.UnitCell(mesh)

inc = ce.InclusionCircle(2, (0.5, 0.5), 0.25)
inc_di = {'circle_inc': inc}
cell.set_append_inclusion(inc_di)

In [5]:
VFS = VectorFunctionSpace(cell.mesh, "CG", 1,
                              constrained_domain=ce.PeriodicBoundary_no_corner(2))
FS = FunctionSpace(cell.mesh, "CG", 1,
                   constrained_domain=ce.PeriodicBoundary_no_corner(2))

# Set materials
E_m, nu_m, Kappa_m = 2e5, 0.4, 7.
# n = 1000
n = 1000  # 13.Jan
E_i, nu_i, Kappa_i = 1000 * E_m, 0.3, n * Kappa_m

mat_m = ma.neo_hook_eap(E_m, nu_m, Kappa_m)
mat_i = ma.neo_hook_eap(E_i, nu_i, Kappa_i)
mat_li = [mat_m, mat_i]

In [6]:
w = Function(VFS)
el_pot_phi = Function(FS)
strain_space_w = TensorFunctionSpace(mesh, 'DG', 0)
strain_space_E = VectorFunctionSpace(mesh, 'DG', 0)

In [7]:
def deform_grad_with_macro(F_bar, w_component):
    return F_bar + grad(w_component)
def e_field_with_macro(E_bar, phi):
    return E_bar - grad(phi)

In [8]:
comp = com.MicroComputation(cell, mat_li,
                        [deform_grad_with_macro, e_field_with_macro],
                        [strain_space_w, strain_space_E])

## Computation with FD

In [9]:
# sample_num = 8
# delta = np.logspace(-2,-4,num=sample_num)

In [9]:
def avg_mer_stress(F_bar, E_bar):
    comp.input([F_bar, E_bar], [w, el_pot_phi])
    comp.comp_fluctuation()
    return comp.avg_merge_stress()

In [16]:
def conv_check_component(label, F_bar, E_bar, compo, delta):
    C_eff_component_FD = np.zeros(shape=(len(delta),6), dtype=float)
    if label is 'F':
        for i, d in enumerate(delta):
            F_minus = deepcopy(F_bar)
            F_minus[compo] = F_bar[compo] - d/2
            F_plus = deepcopy(F_bar)
            F_plus[compo] = F_bar[compo] + d/2

            P_minus = avg_mer_stress(F_minus, E_bar)
            P_plus  = avg_mer_stress(F_plus, E_bar)
            
            C_eff_component_FD[i,:] = (P_plus - P_minus)/d
    elif label is 'E':
        for i, d in enumerate(delta):
            E_minus = deepcopy(E_bar)
            E_minus[compo] = E_bar[compo] - d/2
            E_plus = deepcopy(E_bar)
            E_plus[compo] = E_bar[compo] + d/2

            P_minus = avg_mer_stress(F_bar, E_minus)
            P_plus  = avg_mer_stress(F_bar, E_plus)
            
            C_eff_component_FD[i,:] = (P_plus - P_minus)/d
    else:
        raise Exception('no such field label')
    
    return C_eff_component_FD 

In [17]:
F_bar = [1.1, 0., 0.1, 1.]

E_bar = [0., 0.2]

delta = [0.01, 0.01/2, 0.01/4, 0.01/8]
delta = [0.01/8]

C_eff_component_FD = conv_check_component('F', F_bar, E_bar, 3, delta)

RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to define nonlinear variational problem F(u; v) = 0 for all v.
*** Reason:  Expecting the boundary conditions to to live on (a subspace of) the trial space.
*** Where:   This error was encountered inside NonlinearVariationalProblem.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2019.1.0
*** Git changeset:  74d7efe1e84d65e9433fd96c50f1d278fa3e3f3f
*** -------------------------------------------------------------------------


## Homogenization Method Result

In [10]:
F_bar = [1.1, 0., 0.1, 1.]

E_bar = [0., 0.2]

In [15]:
comp.input([F_bar, E_bar], [w, el_pot_phi])
comp.comp_fluctuation()
comp.avg_merge_stress()

RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to define nonlinear variational problem F(u; v) = 0 for all v.
*** Reason:  Expecting the boundary conditions to to live on (a subspace of) the trial space.
*** Where:   This error was encountered inside NonlinearVariationalProblem.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2019.1.0
*** Git changeset:  74d7efe1e84d65e9433fd96c50f1d278fa3e3f3f
*** -------------------------------------------------------------------------


In [13]:
comp = com.MicroComputation(cell, mat_li,
                        [deform_grad_with_macro, e_field_with_macro],
                        [strain_space_w, strain_space_E])
comp.input([F_bar, E_bar], [w, el_pot_phi])
comp.comp_fluctuation()
C_eff = comp.effective_moduli_2()

fluctuation computation finished
strain computation finished
average merge moduli computation finished


In [13]:
print (C_eff_component_FD)

NameError: name 'C_eff_component_FD' is not defined

In [14]:
C_eff[:,3]

array([  3.23002219e+04,  -9.13009769e+03,  -1.39168000e+03,
         7.36828607e+04,  -1.46915085e-12,   1.05644552e-11])

## Convergence Check

In [17]:
component = C_eff[:,3]

tmp = np.outer(np.ones((len(delta),1)),np.transpose(component))

error = np.linalg.norm(tmp - C_eff_component_FD, axis=1)/np.linalg.norm(component)

NameError: name 'C_eff_component_FD' is not defined

In [61]:
error

array([  9.12302965e-06,   2.28075104e-06,   5.70186024e-07,
         1.42533577e-07])